<a href="https://colab.research.google.com/github/Farshad72/Twitter_Sentiment_Analysis/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, Dataset


In [ ]:
#sentiment_dataset = load_dataset("tweet_eval", "sentiment", split="train[:20%]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
train_set = Dataset.from_file("/content/drive/MyDrive/Twitter-sentiment-analysis/train_data.arrow")
test_set = Dataset.from_file("/content/drive/MyDrive/Twitter-sentiment-analysis/test_data.arrow")
val_set = Dataset.from_file("/content/drive/MyDrive/Twitter-sentiment-analysis/val_data.arrow")

In [5]:
from sklearn.model_selection import train_test_split

In [69]:
len_train= []
len_train_set = [len(train_set['text'][i]) for i in range(train_set.num_rows)]
max(len_train_set)

176

In [ ]:
#new_dataset= sentiment_dataset.train_test_split(test_size=0.4)
#new_dataset_2 = new_dataset['test'].train_test_split(test_size=0.5)
#train_set = new_dataset['train']
#test_set = new_dataset_2['test']
#val_set = new_dataset_2['train']
#print(train_set.num_rows)
#print(test_set.num_rows)
#print(val_set.num_rows)



5473
1825
1825


In [ ]:
#train_set.save_to_disk("/content/twitter-sentiment-analysis/train_set")
#test_set.save_to_disk("/Desktop/test_set")
#val_set.save_to_disk("/Desktop/val_set")



Saving the dataset (0/1 shards):   0%|          | 0/5473 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1825 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1825 [00:00<?, ? examples/s]

In [7]:
#train_set = Dataset.from_file("/content/twitter-sentiment-analysis/train_set")


FileNotFoundError: [Errno 2] Failed to open local file '/content/twitter-sentiment-analysis/train_set'. Detail: [errno 2] No such file or directory

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [37]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [36]:
#from torch.optim.lr_scheduler import CosineAnnealingLR, LinearWarmup
#LinearWarmup
#, CosineAnnealingLR




ImportError: cannot import name 'LinearWarmup' from 'torch.optim.lr_scheduler' (/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py)

In [38]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [39]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [40]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.to(device).tolist())
            actual_labels.extend(labels.to(device).tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [41]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return preds.item()

In [22]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 3
max_length = 128
batch_size = 64
num_epochs = 5
learning_rate = 2e-5

In [23]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_set['text'], train_set['label'], tokenizer, max_length)
val_dataset = TextClassificationDataset(val_set['text'], val_set['label'], tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)
print(device)

cuda


In [44]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1*total_steps, num_training_steps=total_steps)

In [43]:
import time
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print(time.time())
#validation_accuracy = []
#validation_loss = []

for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        val_loss = 0.0
        train_loss = 0.0
        with torch.no_grad():
            for batch in val_dataloader:

              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['label'].to(device)
              outputs = model(input_ids=input_ids, attention_mask=attention_mask)
              loss = nn.CrossEntropyLoss()(outputs, labels)
              val_loss += loss.item()
            for batch in train_dataloader:
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['label'].to(device)
              outputs = model(input_ids=input_ids, attention_mask=attention_mask)
              loss = nn.CrossEntropyLoss()(outputs, labels)
              train_loss += loss.item()
        train_loss /= len(train_dataloader)
        val_loss /= len(val_dataloader)
        print(f"Training Loss: {train_loss:.4f}")  # Print validation loss
        print(f"Validation Loss: {val_loss:.4f}")  # Print validation loss
        accuracy, report = evaluate(model, val_dataloader, device)

        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)
print(time.time())


1711425745.6132193
Epoch 1/5
Training Loss: 0.9485
Validation Loss: 0.9458


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.5775
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       282
           1       0.56      0.76      0.65       835
           2       0.61      0.59      0.60       708

    accuracy                           0.58      1825
   macro avg       0.39      0.45      0.41      1825
weighted avg       0.49      0.58      0.53      1825

Epoch 2/5
Training Loss: 0.9481
Validation Loss: 0.9443


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.5786
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       282
           1       0.56      0.76      0.65       835
           2       0.61      0.59      0.60       708

    accuracy                           0.58      1825
   macro avg       0.39      0.45      0.42      1825
weighted avg       0.49      0.58      0.53      1825

Epoch 3/5


KeyboardInterrupt: 

In [ ]:

print(time.time())
#!pip install nvidia-smi
#import nvidia_smi
#nvidia_smi.nvmlInit()
#handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
#info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
#print(f"GPU memory: {info.total / 1024**2:.2f} MB")

1711314713.074226


In [82]:
######### Test_set accuracy
test_dataset = TextClassificationDataset(test_set['text'], test_set['label'], tokenizer, max_length)
num_row = test_set.num_rows
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
evaluate(model, test_dataloader, device)

(0.6882191780821918,
 '              precision    recall  f1-score   support\n\n           0       0.61      0.56      0.58       280\n           1       0.69      0.68      0.68       833\n           2       0.72      0.74      0.73       712\n\n    accuracy                           0.69      1825\n   macro avg       0.67      0.66      0.67      1825\nweighted avg       0.69      0.69      0.69      1825\n')

In [ ]:
evaluate(model, test_dataloader, device)

(0.6876712328767123,
 '              precision    recall  f1-score   support\n\n           0       0.59      0.67      0.63       270\n           1       0.69      0.69      0.69       831\n           2       0.73      0.70      0.71       724\n\n    accuracy                           0.69      1825\n   macro avg       0.67      0.68      0.68      1825\nweighted avg       0.69      0.69      0.69      1825\n')